In [ ]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp /content/kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download -d ashwingupta3012/human-faces
! kaggle datasets download -d aliasgartaksali/human-and-non-human

100% 1.81G/1.82G [00:13<00:00, 134MB/s]
100% 1.82G/1.82G [00:13<00:00, 146MB/s]
 88% 217M/245M [00:02<00:00, 91.0MB/s]
100% 245M/245M [00:02<00:00, 120MB/s] 


In [ ]:
! unzip human-faces
! unzip human-and-non-human

Streaming output truncated to the last 5000 lines.
  inflating: human-and-non-human/training_set/training_set/humans/Zafarullah_Khan_Jamali_0002.jpg  
  inflating: human-and-non-human/training_set/training_set/humans/Zaini_Abdullah_0001.jpg  
  inflating: human-and-non-human/training_set/training_set/humans/Zakia_Hakki_0001.jpg  
  inflating: human-and-non-human/training_set/training_set/humans/Zalmay_Khalilzad_0001.jpg  
  inflating: human-and-non-human/training_set/training_set/humans/Zarai_Toledo_0002.jpg  
  inflating: human-and-non-human/training_set/training_set/humans/Zavad_Zarif_0001.jpg  
  inflating: human-and-non-human/training_set/training_set/humans/Zdravko_Mucic_0001.jpg  
  inflating: human-and-non-human/training_set/training_set/humans/Zeljko_Rebraca_0001.jpg  
  inflating: human-and-non-human/training_set/training_set/humans/Zelma_Novelo_0001.jpg  
  inflating: human-and-non-human/training_set/training_set/humans/Zeng_Qinghong_0001.jpg  
  inflating: human-and-non-huma

In [ ]:
import tensorflow as tf
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import shutil
from shutil import copyfile

In [ ]:
# folder path
dataset_path = "/content/Humans"

validation_path = os.path.join(os.getcwd(), "human-and-non-human/test_set/test_set/humans")
training_path = os.path.join(os.getcwd(), "human-and-non-human/training_set/training_set/humans")
training_non_human_path = os.path.join(os.getcwd(), "human-and-non-human/training_set/training_set/non-humans")
validation_non_human_path = os.path.join(os.getcwd(), "human-and-non-human/test_set/test_set/non-humans")

dataset_split_size = .9

In [ ]:
# copying humans dataset to training and validation directory
def dataset_split(source, training, validation, split_size):
  files = []
  for filename in os.listdir(source):
    file_path = os.path.join(source, filename)
    # ignoring 0 size file
    if os.path.getsize(file_path) > 0:
      files.append(filename)
    else:
      print("file is zero size, IGNORING")
  
  training_length = int(len(files) * split_size)
  validation_length = int(len(files) - training_length)

  for filename in files[0:training_length]:
    file_path = os.path.join(source, filename)
    destination = os.path.join(training, filename)
    copyfile(file_path, destination)

  for filename in files[training_length:]:
    file_path = os.path.join(source, filename)
    destination = os.path.join(validation, filename)
    copyfile(file_path, destination)

In [ ]:


dataset_split(dataset_path, training_path, validation_path, dataset_split_size)

print(f"There are {len(os.listdir(training_path))} images of human for training")
print(f"There are {len(os.listdir(training_non_human_path))} images of non human for training")
print(f"There are {len(os.listdir(validation_path))} images of human for validating")
print(f"There are {len(os.listdir(validation_non_human_path))} images of non human for validating")

There are 10509 images of human for training
There are 4006 images of non human for training
There are 2137 images of human for validating
There are 1309 images of non human for validating


In [ ]:
def traing_validation_generator(training_path, validation_path):
  train_datagen = ImageDataGenerator(rescale=1.0/255)
  
  train_generator = train_datagen.flow_from_directory(directory=training_path,
                                                      batch_size=5,
                                                      class_mode='binary',
                                                      target_size=(150, 150))

  validation_datagen = ImageDataGenerator(rescale=1.0/255)

  validation_generator = validation_datagen.flow_from_directory(directory=validation_path,
                                                                batch_size=5,
                                                                class_mode='binary',
                                                                target_size=(150, 150))
  
  return train_generator, validation_generator

In [ ]:
train_generator, validation_generator = traing_validation_generator('/content/human-and-non-human/training_set/training_set/', '/content/human-and-non-human/test_set/test_set/')

Found 14514 images belonging to 2 classes.
Found 3445 images belonging to 2 classes.


In [ ]:
def create_model():


  model = tf.keras.models.Sequential([ 
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
  ])

  
  model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.001),
                loss='binary_crossentropy',
                metrics=['accuracy']) 
    

  return model

In [ ]:
model = create_model()

history = model.fit(train_generator,
                    epochs=5,
                    verbose=1,
                    validation_data=validation_generator)

Epoch 1/5


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


  59/2903 [..............................] - ETA: 13:37 - loss: 0.6520 - accuracy: 0.7051

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


2903/2903 [==============================] - 954s 313ms/step - loss: 0.2413 - accuracy: 0.9174 - val_loss: 0.2765 - val_accuracy: 0.9074
Epoch 2/5
2903/2903 [==============================] - 963s 332ms/step - loss: 0.1856 - accuracy: 0.9507 - val_loss: 0.1874 - val_accuracy: 0.9306
Epoch 3/5
2903/2903 [==============================] - 922s 318ms/step - loss: 0.1866 - accuracy: 0.9566 - val_loss: 0.2684 - val_accuracy: 0.9181
Epoch 4/5
2903/2903 [==============================] - 943s 325ms/step - loss: 0.1877 - accuracy: 0.9579 - val_loss: 0.2772 - val_accuracy: 0.9269
Epoch 5/5
2903/2903 [==============================] - 977s 337ms/step - loss: 0.1893 - accuracy: 0.9612 - val_loss: 0.1600 - val_accuracy: 0.9498


In [ ]:
model.save("human_detection_model")

INFO:tensorflow:Assets written to: human_detection_model/assets


In [ ]:
import numpy as np

from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path='/content/' + fn
  img=image.load_img(path, target_size=(150, 150))
  
  x=image.img_to_array(img)
  x /= 255
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])
  
  classes = model.predict(images, batch_size=1)
  
  print(classes)
  


Saving unnamed.jpg to unnamed.jpg
[[0.00274387]]


In [ ]:
!zip -r "/content/human_detection_model.zip" "/content/human_detection_model/"

  adding: content/human_detection_model/ (stored 0%)
  adding: content/human_detection_model/assets/ (stored 0%)
  adding: content/human_detection_model/saved_model.pb (deflated 89%)
  adding: content/human_detection_model/variables/ (stored 0%)
  adding: content/human_detection_model/variables/variables.index (deflated 66%)
  adding: content/human_detection_model/variables/variables.data-00000-of-00001 (deflated 49%)
  adding: content/human_detection_model/keras_metadata.pb (deflated 93%)
